# SQLite Python: Inserting Data
Reference: https://www.sqlitetutorial.net/sqlite-python/

In [ ]:
import pandas as pd
import sqlite3
import sqlalchemy
from IPython.display import display

### Create a new SQLite database
If there is the menu_data.db file, then this function will connect to the database.
If the menu_data.db file doesn't exist, then this function will create a new database file.


In [ ]:
def create_sqlite_database(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(db_file, "connected/created successfully")
        print("SQLite version:", sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    create_sqlite_database(r"menu_data.db")

### Use the create_connection function, the create_table function, and the main function
### to create tables, define attribute domains, and define constrains
Note that, tables are defined and created, don't have to run this chunk of code

In [ ]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

### Create the main function to create tables
def main():
    sql_create_restaurant_table = """ CREATE TABLE IF NOT EXISTS restaurant (
                                        r_id integer PRIMARY KEY,
                                        r_name text NOT NULL
                                    ); """

    sql_create_food_category_table = """CREATE TABLE IF NOT EXISTS food_category (
                                    cat_id integer PRIMARY KEY,
                                    cat_name text NOT NULL
                                );"""

    sql_create_item_info_table = """CREATE TABLE IF NOT EXISTS item_info (
                                    item_id integer NOT NULL,
                                    year integer NOT NULL,
                                    item_name text NOT NULL,
                                    item_description text,
                                    PRIMARY KEY (item_id, year)
                                );"""

    sql_create_nutrition_facts_table = """CREATE TABLE IF NOT EXISTS nutrition_facts (
                                    item_id integer NOT NULL,
                                    year integer NOT NULL,
                                    r_id integer,
                                    cat_id integer,
                                    calories integer,
                                    total_fat integer,
                                    saturated_fat integer,
                                    trans_fat integer,
                                    cholesterol integer,
                                    sodium integer,
                                    potassium integer,
                                    carbs integer,
                                    protein integer,
                                    sugar integer,
                                    dietary_Fiber integer,
                                    PRIMARY KEY (item_id, year),
                                    FOREIGN KEY (item_id) REFERENCES item_info (item_id),
                                    FOREIGN KEY (year) REFERENCES item_info (year),
                                    FOREIGN KEY (r_id) REFERENCES restaurant (r_id),
                                    FOREIGN KEY (cat_id) REFERENCES food_category (cat_id)
                                );"""

    sql_create_combo_facts_table = """CREATE TABLE IF NOT EXISTS combo_facts (
                                    combo_id integer NOT NULL,
                                    item_id integer NOT NULL,
                                    year integer NOT NULL,
                                    cat_id integer,
                                    builds text,
                                    PRIMARY KEY (combo_id, item_id, year),
                                    FOREIGN KEY (item_id) REFERENCES item_info (item_id),
                                    FOREIGN KEY (year) REFERENCES item_info (year),
                                    FOREIGN KEY (cat_id) REFERENCES food_category (cat_id)
                                );"""

    # create a database connection
    conn = create_connection(r"menu_data.db")

    # create tables
    if conn is not None:
        # create tables
        try:
            create_table(conn, sql_create_restaurant_table)
            create_table(conn, sql_create_food_category_table)
            create_table(conn, sql_create_item_info_table)
            create_table(conn, sql_create_nutrition_facts_table)
            create_table(conn, sql_create_combo_facts_table)
        except:
            print("Error! cannot create the database connection.")
        finally:
            if conn:
                conn.close()

if __name__ == '__main__':
    main()

### Testing: check tables

In [ ]:
conn = None
sql = """SELECT name FROM sqlite_master WHERE type='table';"""
sql2 = """SELECT sql
        FROM sqlite_master
        WHERE name = 'nutrition_facts'"""
try:
    conn = sqlite3.connect(r"menu_data.db")
    if conn is not None:
        cursor = conn.cursor()
        cursor.execute(sql)
        print(cursor.fetchall())
        cursor.close()
        conn.close()
except sqlite3.Error as e:
    print(e)

# Exporting pandas DataFrames into SQLite with SQLAlchemy, Appending DataFrame to DB Tables
Reference: https://www.fullstackpython.com/blog/export-pandas-dataframes-sqlite-sqlalchemy.html

In [ ]:
def append_table(db_file, tbl_name, pd_df):
    """
    Use SQLAlchemy to append pandas dataframe to the target table in the SQLite db
    :param db_file: SQLite db file name
    :param tbl_name: SQLite db table name, target table
    :param pd_df: Pandas dataframe name, the exporting df
    :return:
    """
    db_path = 'sqlite:///' + db_file
    engine = sqlalchemy.create_engine(db_path, echo=True)
    sqlite_connection = engine.connect()
    sqlite_table = tbl_name
    pd_df.to_sql(sqlite_table, sqlite_connection, if_exists='append', index = False)

In [ ]:
append_table(db_file="menu_data.db", tbl_name="combo_facts", pd_df=combo_facts)

## The query_as_df function is used to display the SQL query result table

In [5]:
def query_as_df(db_file, sql):
    """
    Display SQLite db query results as a pandas df
    :param db_file: SQLite db file name
    :param sql: the sql query in this form '''query'''
    :return:
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        if conn is not None:
            df = pd.read_sql(sql, con = conn)
            conn.close()
    except sqlite3.Error as e:
        print(e)
    display(df)

In [8]:
### Testing database connection and the target table with imported data
sql = '''
SELECT * FROM food_category;
'''
query_as_df("menu_data.db", sql)

,cat_id,cat_name
0,0,Appetizers & Sides
1,1,Baked Goods
2,2,Beverages
3,3,Burgers
4,4,Desserts
5,5,Entrees
6,6,Fried Potatoes
7,7,Pizza
8,8,Salads
9,9,Sandwiches
